#Import torch & device info


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'avaiable device : %s' % device)
# print(f'avaiable device : {device}')
# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

avaiable device : cuda


 # Prepare data & data loader


In [ ]:
# MNIST dataset
mnist_train = torchvision.datasets.MNIST(root='MNIST_data/',
                                         train=True,
                                         transform=transforms.ToTensor(),
                                         download=True)
mnist_test = torchvision.datasets.MNIST(root='MNIST_data/',
                                        train=False,
                                        transform=transforms.ToTensor(),
                                        download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# dataset loader
batch_size = 100

# drop_lat : drops the last non-full batch of each worker’s dataset replica.
train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size,
                                         shuffle=True)

# Define my Neural Network


Check https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#conv2d

In [ ]:
torch.nn.Conv2d??

In [ ]:
# Define my CNN Model (2 conv layers)
class myNN(torch.nn.Module):

    def __init__(self):
        super(myNN, self).__init__()
        # Torch tensor dim. (bath_size, C, H, W)
        # L1 ImgIn shape=(batch_size, 1,  28, 28)
        #    Conv     -> (batch_size, 32, 28, 28)
        #    Relu     -> (batch_size, 32, 28, 28)
        #    Pool     -> (batch_size, 32, 14, 14)
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # L2 ImgIn shape=(batch_size, 32, 14, 14)
        #    Conv      ->(batch_size, 64, 14, 14)
        #    Relu      ->(batch_size, 64, 14, 14)
        #    Pool      ->(batch_size, 64,  7,  7)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.pool1(out)
        #out = self.pool1(self.relu1(self.conv1(x)))

        out = self.conv2(out)
        out = self.relu2(out)
        out = self.pool2(out) 
        #out = self.pool2(self.relu2(self.conv2(x)))
        # dim : (batch_size, 64, 7, 7)
        
        # Flatten them for FC --> dim : (batch_size, 7*7*64)
        out = out.view(out.size(0), -1)   
        out = self.fc(out) # dim : (batch_size, 10)
        return out

## Using nn.Sequential()

In [ ]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
       # Torch tensor dim. (bath_size, C, H, W)
       # L1 ImgIn shape=(batch_size, 1,  28, 28)
       #    Conv     -> (batch_size, 32, 28, 28)
       #    Relu     -> (batch_size, 32, 28, 28)
       #    Pool     -> (batch_size, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
       # L2 ImgIn shape=(batch_size, 32, 14, 14)
       #    Conv      ->(batch_size, 64, 14, 14)
       #    Relu      ->(batch_size, 64, 14, 14)
       #    Pool      ->(batch_size, 64,  7,  7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

# Set training protocols

In [ ]:
# instantiate CNN model
model = myNN().to(device)

In [ ]:
# hyperparamters
learning_rate = 0.001
training_epochs = 5

# define cost/loss * optmizer
criterion = torch.nn.CrossEntropyLoss().to(device) # Softmax is internally computed.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training myNN

In [ ]:
# train myNN
total_batch = len(train_loader)
print('Learning started.')

for epoch in range(training_epochs):
    running_loss = 0.0
    for data, labels in train_loader:
        # image is already size of (1x28x28), no reshape
        # label is not one-hot encoded
        
        #load the data into the device (GPU)
        data = data.to(device) 
        labels = labels.to(device)

        optimizer.zero_grad() # set gradients to zero
        outputs = model(data) # inference the myNN
        loss = criterion(outputs, labels) #compute the loss
        loss.backward() #back prop
        optimizer.step() #weight update

        running_loss += loss.item() / total_batch

    print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch + 1, running_loss))

print('Learning Fishied.')        


Learning started.
[Epoch:    1] loss = 2.22811856
[Epoch:    2] loss = 1.93482311
[Epoch:    3] loss = 1.29381988
[Epoch:    4] loss = 0.758208605
[Epoch:    5] loss = 0.552785172
Learning Fishied.


# Validate myNN

Refer to torch.max() document. https://pytorch.org/docs/stable/generated/torch.max.html

In [ ]:
correct = 0
total = 0

# Note that when you test the model not train, you set model.eval() mode
model.eval()

with torch.no_grad():
    for images, labels in test_loader:

      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

print('Accuracy of myNN on the test set: %.4f %% ' % (100 * correct/total))

Accuracy of myNN on the test set: 87.3200 % 


# Save the model

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
!ls

MNIST_data  model.ckpt	sample_data


# Practice!!
---


## Make train function 

In [ ]:
# train function

def train_model(train_loader, model):

    print('Learning started.')

    print('Learning Fishied.')        


In [ ]:
#train the model

train_model(train_loader, model)

Learning started.
[Epoch:    1] loss = 0.327809485
[Epoch:    2] loss = 0.315247562
[Epoch:    3] loss = 0.304617805
[Epoch:    4] loss = 0.294749019
[Epoch:    5] loss = 0.286224879
Learning Fishied.


## Make test function

In [ ]:
# MNIST test function

def test_model(test_loader, model):


    print('Accuracy of myNN on the test set: %d %% ' % (100 * correct/total))


In [ ]:
# TEST myNN

test_model(test_loader, model)

Accuracy of myNN on the test set: 92 % 


## Add Batch normalization


In [ ]:
# CNN Model (2 conv layers)
class myNN_BN(torch.nn.Module):

    def __init__(self):
        super(myNN_BN, self).__init__()
        # Torch tensor dim. (bath_size, C, H, W)
        # L1 ImgIn shape=(batch_size, 1,  28, 28)
        #    Conv     -> (batch_size, 32, 28, 28)
        #    BN       -> (batch_size, 32, 28, 28)
        #    Relu     -> (batch_size, 32, 28, 28)
        #    Pool     -> (batch_size, 32, 14, 14)
        #### Write your code ####


        #########################

        # L2 ImgIn shape=(batch_size, 32, 14, 14)
        #    Conv      ->(batch_size, 64, 14, 14)
        #    BN        ->(batch_size, 64, 14, 14)
        #    Relu      ->(batch_size, 64, 14, 14)
        #    Pool      ->(batch_size, 64,  7,  7)

        #### Write your code ####


        #########################


        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)


    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [ ]:
#train the model

train_model(train_loader, model)

Learning started.
[Epoch:    1] loss = 0.278124214
[Epoch:    2] loss = 0.270735431
[Epoch:    3] loss = 0.263705426
[Epoch:    4] loss = 0.257289764
[Epoch:    5] loss = 0.251072471
Learning Fishied.


In [ ]:
#test the model

test_model(test_loader, model)

Accuracy of myNN on the test set: 93 % 
